# テキスト分類のファインチューニング
IMDbデータセット上でDistilBERTをファインチューニングして、映画レビューが肯定的か否定的かを判断するモデルを作ります。

## load dataset

In [ ]:
# !pip install transformers datasets

In [6]:
from datasets import load_dataset

imdb = load_dataset("imdb")

Found cached dataset imdb (/Users/hajime/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1)


  0%|          | 0/3 [00:00<?, ?it/s]

In [7]:
imdb["test"][0]

{'text': 'I love sci-fi and am willing to put up with a lot. Sci-fi movies/TV are usually underfunded, under-appreciated and misunderstood. I tried to like this, I really did, but it is to good TV sci-fi as Babylon 5 is to Star Trek (the original). Silly prosthetics, cheap cardboard sets, stilted dialogues, CG that doesn\'t match the background, and painfully one-dimensional characters cannot be overcome with a \'sci-fi\' setting. (I\'m sure there are those of you out there who think Babylon 5 is good sci-fi TV. It\'s not. It\'s clichéd and uninspiring.) While US viewers might like emotion and character development, sci-fi is a genre that does not take itself seriously (cf. Star Trek). It may treat important issues, yet not as a serious philosophy. It\'s really difficult to care about the characters here as they are not simply foolish, just missing a spark of life. Their actions and reactions are wooden and predictable, often painful to watch. The makers of Earth KNOW it\'s rubbish as 

- text: 映画レビューのテキスト
- label: 否定的なレビューの場合は0、肯定的なレビューの場合は1

## preprocess

1. モデルに先立って、事前学習モデルとセットのトークナイザーをインスタンス化します。
2. テキストをトークン化し、DistilBERTの最大入力長以下になるようにシーケンスを切り詰めるpreprocess_function()という関数を作成します。tructionがその役目です。データセット全体に対して前処理関数を適用するために、map関数を使用します。 `batched=True`を設定すると、map関数を高速化することができます。
3. DataCollatorWithPadding を使用して、データをサンプルするバッチを作成します。また、バッチ内の最も長い要素の長さに合わせてテキストを動的にパディングし、データの長さをそろえます。トークナイザー関数の中で padding=True を設定してテキストを詰めることもできますが、動的なパディングの方がより効率的です。

In [14]:
from transformers import AutoTokenizer
from transformers import DataCollatorWithPadding

# 1
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

# 2
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

tokenized_imdb = imdb.map(preprocess_function, batched=True)

# 3
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Loading cached processed dataset at /Users/hajime/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1/cache-282ea8cadface71d.arrow
Loading cached processed dataset at /Users/hajime/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1/cache-0044d531500dfb33.arrow


  0%|          | 0/50 [00:00<?, ?ba/s]

## Train
1. TrainingArgumentsに学習用ハイパーパラメータを定義する。
2. TrainingArgumentsをmodel、dataset、tokenizer、data_collatorと一緒にTrainerに渡す。
3. train()を呼び出し、モデルのファインチューニングを行う。

In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_imdb["train"],
    eval_dataset=tokenized_imdb["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()